In [2]:
!pip install transformers
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=5b9010ab56e07b99006a3221ffba69d30ae04838bbe5abd600cf1a7bb272f2d7
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [ ]:
import wikipedia
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

# Load Wikipedia articles
topics = ['Artificial intelligence', 'Deep learning', 'Natural language processing']
texts = []
for topic in topics:
    article = wikipedia.page(topic)
    texts.append(article.content)

# Load GPT-2 tokenizer and model
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize texts and prepare for fine-tuning
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()
for epoch in range(3):
    for batch in loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device=device)
        attention_mask = batch[1].to(device=device)
        labels = input_ids.clone().detach()
        labels[labels == tokenizer.pad_token_id] = -100
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

# Generate some text using the fine-tuned model
prompt = "Artificial intelligence is"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device=device)
output = model.generate(input_ids, max_length=100, do_sample=True)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligence is coming to a head of its own after Microsoft unveiled its HoloLens machine-made headsets, called HoloLens, earlier this week as it unveiled its new first-of-its kind gaming headset. HoloLens is a new kind of holographic augmented reality head that works better at rendering objects.

"I can make them look like people's heads," said Dan Hargadar, a Microsoft researcher at Carnegie Mellon who cofounded the company with Hargadar, in


In [3]:
!pip install huggingface-hub
from huggingface_hub import notebook_login

notebook_login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
model.save_pretrained('/content/sample_data/1')

NameError: ignored

Fine tuning single sentence

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Load pre-trained tokenizer and model for question-answering
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

# Define context and question
context = "Astronomy (from Ancient Greek ἀστρονομία (astronomía) 'science that studies the laws of the stars') is a natural science that studies celestial objects and phenomena. It uses mathematics, physics, and chemistry in order to explain their origin and evolution. Objects of interest include planets, moons, stars, nebulae, galaxies, and comets. Relevant phenomena include supernova explosions, gamma ray bursts, quasars, blazars, pulsars, and cosmic microwave background radiation. More generally, astronomy studies everything that originates beyond Earth's atmosphere. Cosmology is a branch of astronomy that studies the universe as a whole.[1]"
question1 = "What studies celestial objects and phenomena"
# Use pipeline to generate answer
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)
result = nlp(question=question1, context=context)
# Print answer
print(result["answer"])

Astronomy


In [9]:
question2="Astronomy uses what to explain their origin?"
question3="What are interests of astronomy?"
result1 = nlp(question=question2, context=context)
result2 = nlp(question=question3, context=context)
print(result1["answer"])
print(result2["answer"])


mathematics, physics, and chemistry
planets, moons, stars, nebulae
